In [ ]:
!pip install Flask line-bot-sdk numpy tensorflow pillow

In [ ]:
import json
from io import BytesIO

from flask import Flask, request, abort
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps
import numpy as np


from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    ImageMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    line_bot_blob_api = MessagingApiBlob(api_client)
model = load_model(env['YOUR_HDF5_FILE'])
labels_file = env.get('YOUR_LABELS_FILE', 'labels.txt')


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )
        

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_message(event):
    message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
        
    # classify
    image = Image.open(BytesIO(message_content))
    r = classify(image)
        
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=r)]
        )
    )


def classify(image):
    image = ImageOps.fit(image, (224, 224))
    image = (np.expand_dims(image, axis=0).astype(np.float32) / 127.5) - 1
    prediction = model.predict(image)
    return labels(np.argmax(prediction))


def labels(i):
    with open(labels_file, 'r', encoding='utf-8') as f:
        row = f.readline()
        while row:
            row = row.strip('\n').split()
            if len(row) > 1 and int(row[0]) == i:
                return row[1]
            row = f.readline()
    return 'unknown'

        
if __name__ == "__main__":
    app.run()